# 1. Importar Dependências

In [ ]:
!pip install opencv-python

In [ ]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

# Import numpy
import numpy as np

import shutil

import random

from math import ceil

# 2. Organizar as pastas 

In [ ]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [ ]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
         !mkdir {IMAGES_PATH}

# 3. Pegar Imagens

In [ ]:
imagens = []

i = 1
for nome_arquivo in os.listdir(IMAGES_PATH):
    FULL_PATH = os.path.join(IMAGES_PATH, nome_arquivo)

    if nome_arquivo.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        try:
            imagem = cv2.imread(FULL_PATH)
            imagens.append(imagem)
            if FULL_PATH == os.path.join(IMAGES_PATH, 'imagem' + str(i) + '.jpg'):
                os.rename(FULL_PATH, os.path.join(IMAGES_PATH, 'imagem' + str(i) + '.jpg'))
            i += 1
        except Exception as e:
            print(f"Erro ao abrir a imagem {caminho_completo}: {str(e)}")

# 4. Pré processamento

## Operações

In [ ]:
def padronizar_dimensoes(imagem, largura, altura):
    return cv2.resize(imagem, (largura, altura))


def normalizar_cores(imagem):
    return cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)


def remover_ruidos(imagem):
    row, col = 1, 1
    return cv2.GaussianBlur(imagem, (row, col), 0)


def augmentacao_dados(imagem):
    return cv2.rotate(imagem, cv2.ROTATE_90_CLOCKWISE)


def ajuste_exposicao_contraste(imagem):
    alpha, beta = 1.0, 0.9
    return cv2.convertScaleAbs(imagem, alpha=alpha, beta=beta)


def equalizacao_histograma(imagem):
    return cv2.equalizeHist(imagem)


def filtragem_bordas(imagem):
    return cv2.Laplacian(imagem, cv2.CV_64F)


def segmentacao(imagem, limiar):
    _, imagem_segmentada = cv2.threshold(imagem, limiar, 255, cv2.THRESH_BINARY)
    return imagem_segmentada


def remover_artefatos(imagem):
    return cv2.medianBlur(imagem, 1)

def aumentar_brilho(imagem):
    brilho = 40
    imagem_brilho = np.where((255 - imagem) < brilho, 255, imagem + brilho)
    
    contraste = 1.2
    imagem_contraste = cv2.convertScaleAbs(imagem_brilho, alpha=contraste, beta=0)

    return imagem_contraste

## Processamento das imagens na lista

In [ ]:
imagens_processadas = []
for imagem in imagens:
    largura, altura = 3560, 3269
    
    # padronizar dimensoes
    imagem = padronizar_dimensoes(imagem, largura, altura)
    # preto e branco
    imagem = normalizar_cores(imagem)
    # desfoque gausiano na imagem
    imagem = remover_ruidos(imagem)
    # rotacionar imagem
    # imagem = augmentacao_dados(imagem)
    # contraste
    imagem = ajuste_exposicao_contraste(imagem)
    # equalizar os valores dos pixels na imagem
    imagem = equalizacao_histograma(imagem)
    # outro desfoque, so q mediano
    imagem = remover_artefatos(imagem)
    # atribui um valor binario a cada pixel dependendo de um threshhold
    # imagem = segmentacao(imagem, 164)
    # deixa os pixels q representam as "bordas" de um objeto
    # imagem = filtragem_bordas(imagem)

    imagens_processadas.append(imagem)

## Salvar imagens

In [ ]:
PROCESSED_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'processedimages')

In [ ]:
if not os.path.exists(PROCESSED_PATH):
    if os.name == 'posix':
        !mkdir -p {PROCESSED_PATH}
    if os.name == 'nt':
        !mkdir {PROCESSED_PATH}

In [ ]:
i = 1
for img in imagens_processadas:
    IMAGE_PATH = os.path.join(PROCESSED_PATH, 'imagem_processada_' + str(i) + '.jpg')
    cv2.imwrite(IMAGE_PATH, img)
    i += 1

# 5. Image Labelling

In [ ]:
!pip install --upgrade pyqt5 lxml

In [ ]:
LABELING_PATH = os.path.join('Tensorflow', 'labelimg')

In [ ]:
if not os.path.exists(LABELING_PATH):
    !mkdir {LABELING_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELING_PATH}

In [ ]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELING_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
!cd {LABELING_PATH} && python labelImg.py

# 6. Popular imagens processadas com imagens alteradas

## Criar um conjunto de imagens aleatorias para seleção

In [ ]:
ratio = 0.9
num_augmented = ceil(len(os.listdir(PROCESSED_PATH))) * ratio

In [ ]:
def escolher_numeros_aleatorios(seed, minimo, maximo, qtd_numeros):
    random.seed(seed)
    if qtd_numeros > (maximo - minimo + 1):
        return "Impossível selecionar essa quantidade de números únicos dentro do intervalo."
    
    numeros_escolhidos = []
    
    while len(numeros_escolhidos) < qtd_numeros:
        novo_numero = random.randint(minimo, maximo)
        if novo_numero not in numeros_escolhidos:
            numeros_escolhidos.append(novo_numero)
    
    return numeros_escolhidos

numeros_aleatorios = escolher_numeros_aleatorios(42, 0, len(os.listdir(IMAGES_PATH)) - 1, num_augmented)
print(numeros_aleatorios)

## Criar uma pasta para as imagens alteradas

In [ ]:
AUGMENTED_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'augmentedimages')

In [ ]:
start = len(imagens_processadas) + 1

In [ ]:
if not os.path.exists(AUGMENTED_PATH):
    if os.name == 'posix':
        !mkdir -p {AUGMENTED_PATH}
    if os.name == 'nt':
        !mkdir {AUGMENTED_PATH}

In [ ]:
i = 0
for idx in numeros_aleatorios:
    IMAGE_PATH = os.path.join(PROCESSED_PATH, 'imagem_processada_' + str(i + start) + '.jpg')
    cv2.imwrite(IMAGE_PATH, aumentar_brilho(imagens_processadas[idx]))
    i += 1

## Copiar xml correspondentes

In [ ]:
source_directory = PROCESSED_PATH
destination_directory = PROCESSED_PATH

In [ ]:
files = os.listdir(source_directory)
xml_files = [file for file in files if file.endswith('.xml')]

i = 0
for idx in numeros_aleatorios:
    selected = [file for file in files if file.endswith('_' + str(idx + 1) + '.xml')]
    file = selected[0]
    source_file_path = os.path.join(source_directory, file)
    new_file_name = f"imagem_processada_{i + start}.xml"
    destination_file_path = os.path.join(destination_directory, new_file_name)
    shutil.copyfile(source_file_path, destination_file_path)
    i += 1

# 7. Move them into a Training and Testing Partition

## Criar caminhos para as pastas

In [ ]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

## Selecionar arquivos aleatórios para teste e treino

In [ ]:
files = os.listdir(PROCESSED_PATH)

In [ ]:
def copy_img_xml(destination_path, array_idx):
    for idx in array_idx:
        selected_img = [file for file in files if file.endswith(f'_{idx + 1}.jpg')][0]
        selected_xml = [file for file in files if file.endswith(f'_{idx + 1}.xml')][0]

        source_file_path = os.path.join(PROCESSED_PATH, selected_img)
        destination_file_path = os.path.join(destination_path, selected_img)
        shutil.copyfile(source_file_path, destination_file_path)
        
        source_file_path = os.path.join(PROCESSED_PATH, selected_xml)
        destination_file_path = os.path.join(destination_path, selected_xml)
        shutil.copyfile(source_file_path, destination_file_path)

## Separar grupo de imagens

In [ ]:
max = len(files) // 2 - 1
train_ratio = 0.9
count = ceil((len(files) // 2) * train_ratio)
train_idx = escolher_numeros_aleatorios(420, 0, max, count)
test_idx = [x for x in range(len(files) // 2) if x not in train_idx]

In [ ]:
copy_img_xml(TRAIN_PATH, train_idx)
copy_img_xml(TEST_PATH, test_idx)

# 8. Comprimir as pastas do modelo

In [ ]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}